**Assignment 1**

## CSCI E-118 Introduction to Blockchain and Bitcoin

### Part 2: A Simple Blockchain

In this assignment we will create a simple blockchain. It will be different from the Bitcoin or Ethereum blockchains, but will illustrate many of the concepts we've been discussing so far. Much of the code is written for you, but you will be asked to fill in some of the missing pieces.

First we start by importing the packages we will be using (make sure you have the kernel set to **block_env**). The **ecdsa** library was included in the **installations.sh** script.

In [1]:
import os
from ecdsa import SigningKey, VerifyingKey, SECP256k1
from ecdsa.util import randrange_from_seed__trytryagain
from hashlib import sha256
import time
import calendar

#### Primer on hashing and public-private keys with Python

There are two main cryptographic primatives we're concerned with. The first is cryptograph hash functions. The second is Public-Private key cryptography, specifically for the purpose of digital signatures.

For cryptographic hash functions, Bitcoin primarily uses $\text{SHA-256}^2$, i.e. `sha256( sha256(x) )`. Ethereum primarily uses $\text{KECCAK-256}$, otherwise known as $\text{ETH-hash}$ to dissambiguate it from $\text{SHA-3}$. To keep it simple, we'll just use $\text{SHA-256}$ (without squaring it.

The `hashlib` library that comes with python will allow us to do this. We'll try hashing a simple message.

In [2]:
sha256('hello')

TypeError: Unicode-objects must be encoded before hashing

The reason this doesnt work is because the string must be byte encoded. We can do this by prefixing a `b` in front of the string, or by using the `str.encode()` method.

In [3]:
assert b'hello' == 'hello'.encode()

hello_hash = sha256(b'hello')

hello_hash

<sha256 HASH object @ 0x7fba00321850>

It worked, but what we got back was an object. What do we do with it? Here are some options:

In [4]:
[prop for prop in dir(hello_hash) if not prop.startswith('_')]

['block_size', 'copy', 'digest', 'digest_size', 'hexdigest', 'name', 'update']

`digest` and `hexdigest` both look promising. Let's try `digest`...

In [5]:
hello_hash.digest()

b',\xf2M\xba_\xb0\xa3\x0e&\xe8;*\xc5\xb9\xe2\x9e\x1b\x16\x1e\\\x1f\xa7B^s\x043b\x93\x8b\x98$'

What we got back is a bunch of bytes representing the hash.

In [6]:
type(hello_hash.digest())

bytes

In fact we got exactly $32$ byes:

In [7]:
hello_hash.digest_size

32

$32$ byes is equal to $32 * 8 = 256$ bits, hence the $256$ in $\text{SHA-256}$.

In [8]:
32 * 8

256

But looking at the bytes isn't very readable for a human. It's better if we represent it in hexadecimal. We can convert a bytestring to a hexstring using `.hex()`.

In [9]:
hello_hash.digest().hex()

'2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824'

And that is exactly what we get when using `hexdigest()`.

In [10]:
hello_hash.digest().hex() == hello_hash.hexdigest()

True

What else can we hash? Well we can hash a concatenation of strings. We might try this approach:

In [11]:
string1 = 'foo'
string2 = 'bar'
string3 = 'baz'

string_to_hash = string1+string2+string3
string_to_hash

'foobarbaz'

We concatenated our strings, now we encode it and hash the result:

In [12]:
sha256(string_to_hash.encode()).hexdigest()

'97df3588b5a3f24babc3851b372f0ba71a9dcdded43b14b9d06961bfc1707d9d'

Alternatively, we could try this approach:

In [13]:
m = sha256()
m.update(string1.encode())
m.update(string2.encode())
m.update(string3.encode())
m.hexdigest()

'97df3588b5a3f24babc3851b372f0ba71a9dcdded43b14b9d06961bfc1707d9d'

We got the same result. Since we're using a cryptographic hash function, we know that is extremely unlikely to be coincidence (hash collison). The outputs are equivilent because the approaches are equivilent. You can use either way, but I prefer the second approach, using the `.update()` method.

Now for the second cryptographic primative.

The [ecdsa](https://github.com/tlsfuzzer/python-ecdsa) package provides a library for producing key-pairs using the Elliptic-Curve Digital Signature Algorithm. The Bitcoin and Ethereum blockchains use **secp256k1**, which is a certain kind of elliptic curve with certain intial parameters (read the spec if you're interested). We'll use it as well.

Recall, we're interested in digital signatures. The goal is to generate a private key, for signing messages, and a public key for verifying the digital signature against the original message.

Here's how we achieve that:

In [14]:
message = b'message' # make sure to byte-encode the message

sk = SigningKey.generate() # our private key
vk = sk.verifying_key # our public key
signature = sk.sign(message) # generate the digital signature
vk.verify(signature, message) # verify the signature agains the original message

True

Easy enough, but there's a few issues. Firstly, the default curve used is `NIST192p`. We want `SECP256k1`. That's easily solved.

In [15]:
message = b'message' # make sure to byte-encode the message

sk = SigningKey.generate(curve=SECP256k1) # our private key
vk = sk.verifying_key # our public key
signature = sk.sign(message) # generate the digital signature
vk.verify(signature, message) # verify the signature agains the original message

True

Let's inspect the keys.

In [16]:
sk

In [17]:
vk

VerifyingKey.from_string(b'\x02\xb3D\x16\xac\x04\x85 i\xdd)\x84\xfd\x90\xa1x\xaf\xf1\xcf\x91\x162\x98"\xee\xa2\xc5\xcf\xa8\x92\r7\xb1', SECP256k1, sha1)

In [18]:
signature

b'\xede%\x97\xd3T\xcc}7\x9f\x9e\x05\x06\xc0v\xe0\x06\x99t0\x06\\&<\xadC\xaa[\x1aN\xb7\x92h\x05YXP=\x87\xd2\xb8\x89h\xc3n\xf8\xd2\xa2\x9f\xd5k \x94\x85RSUa\x16\x97?h]\xab'

Okay, not very readable. Let's try again:

In [19]:
sk.to_string()

b'\x8a\x8b\xe6\x08#\t\x8eF,\xc3\xf1O\xcf"\xab\x0e\xde\x9d\xc1\xc7 \x0e\xaf\xb60&\x11V\xf8\xd1\xea\x1d'

In [20]:
vk.to_string()

b'\xb3D\x16\xac\x04\x85 i\xdd)\x84\xfd\x90\xa1x\xaf\xf1\xcf\x91\x162\x98"\xee\xa2\xc5\xcf\xa8\x92\r7\xb1\x9b//\xf4xX0]F\xff$\xd0\x147\x95\x8c\x81r\x8b\x8e\x1f\x1e\xe3\xc1&\xe2c\xe9\xe0cWD'

We know what to do with that...

In [21]:
sk.to_string().hex()

'8a8be60823098e462cc3f14fcf22ab0ede9dc1c7200eafb630261156f8d1ea1d'

In [22]:
vk.to_string().hex()

'b34416ac04852069dd2984fd90a178aff1cf9116329822eea2c5cfa8920d37b19b2f2ff47858305d46ff24d01437958c81728b8e1f1ee3c126e263e9e0635744'

In [23]:
signature.hex()

'ed652597d354cc7d379f9e0506c076e006997430065c263cad43aa5b1a4eb79268055958503d87d2b88968c36ef8d2a29fd56b2094855253556116973f685dab'

The second issue is that it's generating random keys, if you run the code again you get different values. In general that's a good thing. But for this homework I want you to get very specific values so I can check your results. Here's how we specify a seed value to deterministically generate a key-pair. First we'll try a random seed. We want the seed to be of length $32$:

In [24]:
SECP256k1.baselen

32

Then we can use this to generate a secrete exponent. Generating a secret exponent relies on the seed and the `order` of the curve we're using. We use the function `randrange_from_seed__trytryagain()`. The details are bit out of scope, but you can read the specifications for SECP256k1 if you're interested in what the `order` refers to. Look ath the [ecdsa](https://github.com/tlsfuzzer/python-ecdsa) github for details on `randrange_from_seed__trytryagain`.

We call it an exponent, but it's really reffering to the number of multiplications performed on the elliptic curve (as discussed in lecture). With a given secret exponent, we get a specific signing key (private key) and verifying key (public key). We should also use `sign_deterministic()` instead of `sign()`.

In [25]:
seed = os.urandom(SECP256k1.baselen)
print( f'seed: {seed.hex()}' )

secexp = randrange_from_seed__trytryagain(seed, SECP256k1.order)
print( f'secret exponent: {secexp}')

private_key = SigningKey.from_secret_exponent(secexp, curve=SECP256k1)
print( f'private key: {private_key.to_string().hex()}' )

public_key = private_key.verifying_key
print( f'public key: {public_key.to_string().hex()}' )

signature = private_key.sign_deterministic(b'message')
print(f'signature: {signature.hex()}')

assert public_key.verify(signature, b'message')

seed: 546723860617210e7794d7e35e1fface551b895d87ee1005ca3579557f87eb59
secret exponent: 23444613528565270364129993989543089886159056883507596932102794021379756409855
private key: 33d52e5701a5f0c9daba7f804da754cc7160505929da53d70ef1c72528ca43ff
public key: befe4ffcf0f885443e1f0659a2ad886e911ee75ac0ade61eb2468fb880956f0dfdfe121d67eaf3476fb36b6b2838ab2a131c839847f8ed91b2ca3b6cbdc15696
signature: 284cb6f424768c81dcd0423704a928cb0a50dd3a1e21c3d572f8ebc0e6f7cf52776c5c5caba04db6948bb841d4b15e41fa42a067eee1092d8f553728c4c249db


Okay, but if we start with a random seed, we'll get a different set of outputs each time. Let's try running through the process with a specific seed value.

In [26]:
seed = bytes.fromhex('6daa908eb3d309183a9f4789a00f51b18542da1417be0cdd799081228236dae9')
print( f'seed: {seed.hex()}' )

secexp = randrange_from_seed__trytryagain(seed, SECP256k1.order)
print( f'secret exponent: {secexp}')

private_key = SigningKey.from_secret_exponent(secexp, curve=SECP256k1)
print( f'private key: {private_key.to_string().hex()}' )

public_key = private_key.verifying_key
print( f'public key: {public_key.to_string().hex()}' )

signature = private_key.sign_deterministic(b'message')
print(f'signature: {signature.hex()}')

assert public_key.verify(signature, b'message')

seed: 6daa908eb3d309183a9f4789a00f51b18542da1417be0cdd799081228236dae9
secret exponent: 85314299752260761361628249051057399690231672481648748497211137228769388769381
private key: bc9e2eb5d3a6857f52803f57750a2e7c9a75e8d1a14784e61f38845ecc6db065
public key: 57b28b2862c10eb35a0cc9799543ad75ed88c1151396993e5d43bb2f1851acd2665e92bff0daf899374c1bf6723b84d112bdfed23ace5fa2f00e629080f7d638
signature: 4434c50c818bbc7baa44e75553afea448608472051b27dbb69ae47f193004c3652113d6a61793e2700fa6aa805249804e0b1c55caa6751511d5c975c8694c0b8


If we do it again, we'll get the same results:

In [27]:
seed = bytes.fromhex('6daa908eb3d309183a9f4789a00f51b18542da1417be0cdd799081228236dae9')
print( f'seed: {seed.hex()}' )

secexp = randrange_from_seed__trytryagain(seed, SECP256k1.order)
print( f'secret exponent: {secexp}')

private_key = SigningKey.from_secret_exponent(secexp, curve=SECP256k1)
print( f'private key: {private_key.to_string().hex()}' )

public_key = private_key.verifying_key
print( f'public key: {public_key.to_string().hex()}' )

signature = private_key.sign_deterministic(b'message')
print(f'signature: {signature.hex()}')

assert public_key.verify(signature, b'message')

seed: 6daa908eb3d309183a9f4789a00f51b18542da1417be0cdd799081228236dae9
secret exponent: 85314299752260761361628249051057399690231672481648748497211137228769388769381
private key: bc9e2eb5d3a6857f52803f57750a2e7c9a75e8d1a14784e61f38845ecc6db065
public key: 57b28b2862c10eb35a0cc9799543ad75ed88c1151396993e5d43bb2f1851acd2665e92bff0daf899374c1bf6723b84d112bdfed23ace5fa2f00e629080f7d638
signature: 4434c50c818bbc7baa44e75553afea448608472051b27dbb69ae47f193004c3652113d6a61793e2700fa6aa805249804e0b1c55caa6751511d5c975c8694c0b8


Nice. Looks like we have all the cryptographic tools we need. Now let's get started with making a blockchain. Make sure to read carefully. You'll need to provide the necessary code in places to get it to work.

#### Building a wallet

A wallet holds your cryptographic keys. We'll also have it implement some methods for digitally signing messages, and verifying those signatures.

**Fill in the code where you see "TODO"**.

Make sure you're using the deterministic variant for key generation.

In [423]:
class Wallet:
    """
    A class that represents an individual account.
    
    Attributes
    ----------
    seed : length 32 hexadecimal string
        represents 16 bytes, used in deterministically generating key-pair
    sexexp : int
        secret exponent; used in determining ecdsa key pair; generated from seed
    private_key: ecdsa.keys.SigningKey
        generated from secret exponent; used to sign messages
    public_key: ecdsa.keys.VerifyingKey
        generated from private_key; used to verify digital signatures sined by private_key
    """
    
    def __init__(self, seed_hex=None, display=False):
        """
        initializes wallet
        
        Parameters
        ----------
        seed_hex : length 32 hexadecimal string, optional
            represents 16 bytes, used in deterministically generating key-pair (default is random hex value)
        """
        if seed_hex:
            self.seed = bytes.fromhex(seed_hex)
        else:
            self.seed = os.urandom(SECP256k1.baselen)
            
        """
        TODO: Your Code Here
        """
            
        self.display_creation(display)
        
    def sign_message(self, message, display=False):
        """
        digitally sign a message using ecdsa (SECP256k1)
        
        Parameters
        ----------
        message : string
            any arbitrary string to digitally sign
        """
        
        """
        TODO: Your Code Here
        """
        
        if display:
            print(f'signature: {signature.hex()}')
            
        return signature
    
    def verify_signature(self, signature, message, display=False):
        """
        verify a given digital signature against the message it represents
        
        Parameters
        ----------
        signature : bytes
            byte encoded digital signature, signed by private_key
        message : string
            arbitrary string that the signature is representing
        """
        
        """
        TODO: Your Code Here
        """
        
        if display:
            print(f'message: {message}, actual sig: {self.sign_message(message, display).hex()}, given sig: {signature.hex()}')
            
        return is_valid
    
    def display_creation(self, display):
        if display:
            print( f'Wallet seed: {self.seed.hex()}' )
            print( f'Wallet secret exponent: {self.secexp}')
            print( f'Wallet private key: {self.private_key.to_string().hex()}' )
            print( f'Wallet public key: {self.public_key.to_string().hex()}' )

Run the code below. All assertions should pass.

In [29]:
test_wallet = Wallet(seed_hex='6daa908eb3d309183a9f4789a00f51b18542da1417be0cdd799081228236dae9', display=True)

assert test_wallet.private_key.to_string().hex() == 'bc9e2eb5d3a6857f52803f57750a2e7c9a75e8d1a14784e61f38845ecc6db065'
assert test_wallet.public_key.to_string().hex() == '57b28b2862c10eb35a0cc9799543ad75ed88c1151396993e5d43bb2f1851acd2665e92bff0daf899374c1bf6723b84d112bdfed23ace5fa2f00e629080f7d638'

Wallet seed: 6daa908eb3d309183a9f4789a00f51b18542da1417be0cdd799081228236dae9
Wallet secret exponent: 85314299752260761361628249051057399690231672481648748497211137228769388769381
Wallet private key: bc9e2eb5d3a6857f52803f57750a2e7c9a75e8d1a14784e61f38845ecc6db065
Wallet public key: 57b28b2862c10eb35a0cc9799543ad75ed88c1151396993e5d43bb2f1851acd2665e92bff0daf899374c1bf6723b84d112bdfed23ace5fa2f00e629080f7d638


In [30]:
test_sig = test_wallet.sign_message('hello world', display=True)

assert test_sig.hex() == '9bfced9bf3c362c1620fea4450435aa28b352fd7fb15392ca289cf1c0f9b0d9ecaf2ca82bff7fdccb44e4876870efa3dfde78eb900b6e2229da93433b326b700'

signature: 9bfced9bf3c362c1620fea4450435aa28b352fd7fb15392ca289cf1c0f9b0d9ecaf2ca82bff7fdccb44e4876870efa3dfde78eb900b6e2229da93433b326b700


Here's what happens if we try to verify against a bogus digital signature. Below I create a bad signature (notice I've changed the first to digits to '00'.

In [32]:
bad_sig = bytes.fromhex('00fced9bf3c362c1620fea4450435aa28b352fd7fb15392ca289cf1c0f9b0d9ecaf2ca82bff7fdccb44e4876870efa3dfde78eb900b6e2229da93433b326b700')

In [36]:
try:
    test_verify_fail = test_wallet.verify_signature(bad_sig, 'hello world', display=True)
except:
    print('Verification failed (on purpose)')

Verification failed (on purpose)


Okay, finally lets create wallets for Alice, Bob, and Carol. We'll be using them later.

In [78]:
alice = Wallet(seed_hex='7ef6f93701e5ca282c5680f2d39629afafaa00bf1a1cb8bf5a0f20d19c75c39b', display=True)
bob = Wallet(seed_hex='1d08199699f5d877efee7448b3c9eb7bac49a5689b28c92ecaf7ddebca375ae3')
carol = Wallet(seed_hex='609b1379370495999c9fe7fc9aa59cca39b26a2b274a1a97eef4bab24ef760e4')

Wallet seed: 7ef6f93701e5ca282c5680f2d39629afafaa00bf1a1cb8bf5a0f20d19c75c39b
Wallet secret exponent: 34858948105046340472739633959283104990204816252169379477184521129263669664787
Wallet private key: 4d1177272d648729dca17b4cc053f7c4a04b04a99b4fd8cc909bb95786175c13
Wallet public key: 8ec3a2d3a8b406879ad54e78c83aca27c2875972cd48a922938a1e8b31d00a6db7e3fd2dc35cddc91fdba2f6c2d19795a375a0041026bcfce49fb15ef7841815


#### Storing Wallets in an Account Regsitry

Bitcoin uses the **UTXO** model, which stands for unspent transaction output. It's discussed in [Chapter 6 of Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc#transaction-outputs-and-inputs), but we haven't discussed it in class yet. We will, but for now I'll give the very quick summary, using Alice as an example.

Bitcoin doesn't store a central account registry, where everybody's accounts and their associated balances are stored. In theory you can Alice's account balance by starting at the first block and iterating through every transaction ever recorded where Alice's public key occurs. You sum up all of the Bitcoin Alice ever received, and you subtract all of the Bitcoin she ever spent. The total is her balance. In reality, Alice has a piece of software called a wallet that keeps track of her balance. In order for others to verify this, when she makes a transaction, she specifies the locations of her unspent funds. The verification process is then just making sure those funds are indeed unspent. We'll go into more detail about this when we discuss Bitcoin Scripts in class.

Ethereum doesn't use the **UTXO** model. It uses an account model. But we're not implementing anything like Ethereum here. What we're doing is implementing something sort of like a simple version of Bitcoin, but we're going to use the account model instead of the **UTXO** model. The reason for this is just to save you time coding and to keep this assignment from growing too large.

So what we'll do is define an `Account_Registry`. It simply keeps a dictionary of public keys. Note that I'm not keeping `public_key` (i.e. `ecdsa.keys.VerifyingKey`) objects, but the hexstring representations of them (i.e. `VerifyingKey.to_string().hex()`.

The values are dictionaries containing a `name`, `balance` and `wallet`. The `name` is just a string specifying the name of the account holder. It's just for convinience. It's hard to tell who is who when just reading hexadecimal strings, so we'd like to query their name, like 'alice'. The `balance` is the amount of currency they currently have. The `wallet` is a `Wallet` object. Remember, the `Wallet` contains bot of the keys. Obviously this wouldn't work in a real blockchain, since we don't make private keys available. It's just for our convinience. We'll pretend no one  on the network actually has this data available to them.

You don't have to code anything here. I've provided the full class. Do look over the code though.

In [39]:
class Account_Registry:
    """
    A class that represents a collection of accounts, public keys associated with a name, balance, and wallet.
    
    Attributes
    ----------
    accounts : dict
        dictionary of accounts; key is hex string representation of public_key; value is dictionary containing:
            name : string
                arbitrary pseudonym of wallet holder
            balance : int
                current balance
            wallet : Wallet
                wallet of account holder
    """
    def __init__(self, accounts=[]):
        """
        initializes accounts dict
        
        Parameters
        ----------
        accounts : list[ ( string, Wallet ) ], optional
            list of tuples containing the account pseudonym and the wallet object
        """
        self.accounts = dict()
        
        for account in accounts:
            pseudonym, wallet = account
            self.register_account(wallet, pseudonym)
    
    def register_account(self, wallet, pseudonym):
        """
        registers account by placing in accounts dict
        
        Parameters
        ----------
        wallet : Wallet
            wallet of account holder
        pseudonym : string
            arbitrary name of account holder
        """
        if not isinstance(wallet, Wallet):
            raise Exception(f'wallet "{wallet}" is not of type "Wallet"')
        
        public_key = wallet.public_key.to_string().hex()
        if not public_key in self.accounts.keys():
            self.accounts[public_key] = {
                'name': pseudonym,
                'balance': 0,
                'wallet': wallet
            }
        else:
            raise Exception(f'Account {public_key} already registered')
            
    def get_account(self, public_key):
        """
        retrieve account from public key of account holder
        
        Parameters
        ----------
        public_key : ecdsa.keys.VerifyingKey
            public key of account holder
        """
        public_key = public_key.to_string().hex()
        if public_key in self.accounts.keys():
            return self.accounts[public_key]
        else:
            raise Exception(f'Account {public_key} is not registered')
    
    def get_wallet(self, public_key):
        """
        retrieve wallet of account holder
        
        Parameters
        ----------
        public_key : ecdsa.keys.VerifyingKey
            public key of account holder
        """
        account = self.get_account(public_key)
        return account['wallet']
            
    def get_balance(self, public_key):
        """
        retrieve current balance of account holder
        
        Parameters
        ----------
        public_key : ecdsa.keys.VerifyingKey
            public key of account holder
        """
        account = self.get_account(public_key)
        return account['balance']
            
    def get_name(self, public_key):
        """
        retrieve psuedonym of account holder
        
        Parameters
        ----------
        public_key : ecdsa.keys.VerifyingKey
            public key of account holder
        """
        account = self.get_account(public_key)
        return account['name']

Let's register three of our accounts (we'll leave out Nakamoto for now):

In [40]:
account_registry = Account_Registry( accounts=[('alice', alice), ('bob', bob), ('carol', carol)] )

We can see Alice is registered, and her balance is set to $0$.

In [47]:
account_registry.get_name(alice.public_key), account_registry.get_balance(alice.public_key)

('alice', 0)

#### Timestamps

Timestamps are used a lot. We use them to track when transactions were made and when blocks were created. We can use a lot of formats for timestamps, but the ubiquitous format is Unix time. In homework 0, we used `time.time()` to record the amount of seconds between runs of `FactorialMemo`. What is the output of `time.time()`?

In [48]:
time.time()

1612791786.5386932

Well it's the current time, but in what units? The units are seconds. Specifically, the number of seconds since the **epoch**. The epoch is the beginning of all time (from the perspective of Unix). It's set to January 1st, 1970, 00:00:00 (UTC).

In [49]:
time.gmtime(0)

time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=1, tm_isdst=0)

We're going to be using Unix time for our timestamps.

Now, when we perform hashes of transactions and blocks, we include the timestamp. That's no good. When you call `time.time()` it will give you a different value depending on when you call it, which means you'll get a different hash. I want you to get a specific hash, so we're going to take a more deterministic approach.

We'll create our own starting point, and call it **genesis**. It's defined to be January 25th, 2021, 00:00:00 (UTC). January 25th was the first day of class, so it's fitting I think. **genesis** is exactly `1611532800` seconds from the **epoch**. Every timestamp we use will be a specific number of seconds from the **genesis**, i.e. `1611532800 + seconds_from_genesis`. That way I know you're getting specific values for your timestamps.

I've created a function below called `create_timestamp` that does this for us. Again you don't have to do anything here, just look at the code and understand what it's doing.

In [71]:
def create_timestamp(days_since_genesis=0,
                     hours_since_genesis=0,
                     minutes_since_genesis=0,
                     seconds_since_genesis=0,
                     display=False):
    """
    create a timestamp (unix time: seconds since epoch) by specifying time since genesis
    genesis is defined to be Monday, January 25, 2021, midnight, UTC
    
    Parameters
    ----------
    days_since_genesis : int, optional
        number of days since genesis (default 0)
    hours_since_genesis : int, optional
        number of hours since genesis (default 0)
    minutes_since_genesis : int, optional
        number of minutes since genesis (default 0)
    seconds_since_genesis : int, optional
        number of seconds since genesis (default 0)
    """

    # genesis time set to Monday, January 25, 2021, midnight of the first day of class, UTC
    genesis_time_struct = time.struct_time((2021, 1, 25, 0, 0, 0, 0, 25, 0))
    genesis_timestamp = calendar.timegm(genesis_time_struct)
    genesis_time_ascii = time.asctime(genesis_time_struct)

    seconds_since_genesis += minutes_since_genesis * 60
    seconds_since_genesis += hours_since_genesis * 60 * 60
    seconds_since_genesis += days_since_genesis * 60 * 60 * 24

    new_timestamp = genesis_timestamp + seconds_since_genesis
    new_time_struct = time.gmtime(new_timestamp)
    new_time_ascii = time.asctime(new_time_struct)

    if display:
        print(f'genesis time: {genesis_time_ascii}, timestamp: {genesis_timestamp}')
        if genesis_timestamp != new_timestamp:
            print(f'new time: {new_time_ascii}, timestamp: {new_timestamp}')

    return new_timestamp

If we call `create_timestamp` with the default values, we'll get the **genesis** time. If we pass in some values, we'll get some number of seconds since **genesis**. We'll define some timesamps that will be useful later.

In [75]:
genesis_timestamp = create_timestamp(0,0,0,0, display) # genesis timestamp: 1611532800
tx1_ts = create_timestamp(7,0,0,0) # timestamp for transaction 1, exactly 7 days from genesis
tx2_ts = create_timestamp(7,0,0,1) # timestamp for transaction 2, 7 days and 1 second from genesis
b1_ts = create_timestamp(7,0,1,0) # timestamp for block 1, 7 days and 1 minute from genesis

genesis time: Mon Jan 25 00:00:00 2021, timestamp: 1611532800


#### Hashable Objects

There are two types of objects we're going to be taking hashes of: transactions and objects. Transaction hashes are the sha256 hash of the transaction data. Block hashses are the sha256 hash of the block data. Therefore we'll implement a class called `Hashable` which has some common attributes and methods between both transaction object and block objects.

`Hashable` has two attributes in particular that are common between blocks and transactions. The `hash_val` attribute is the hash (in hexstring format, i.e. using `hexdigest()`). The timestamp is what we discussed above, using Unix time.

I've implemented it below. You don't need to code anything here either (don't worry you will soon enough).

In [73]:
class Hashable:
    """
    A class that represents a hashable object. Meant to be subclassed. Hashes with sha256.
    
    Attributes
    ----------
    hash_val : length 64 string encoding of hexadecimal number
        a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
    timestamp : int
        timestamp of object creation in Unix time (seconds since epoch)
    obj_type
        type of subclass extending Hashable
    """
    def __init__(self, timestamp=None):
        """
        initializes the hashable object
        
        hash_val is set to an empty string
        obj_type is set to name of class extending Hashable
        
        Parameters
        ----------
        timestamp : int, optional
            Unix time (seconds since epoch) representing time when object was created (default is current time)
        """
        self.hash_val = ''
        self.timestamp = self.set_timestamp(timestamp)
        self.obj_type = self.__class__.__name__
        
    def set_timestamp(self, timestamp):
        """
        sets the timestamp attribute
        
        Parameters
        ----------
        timestamp: int
            Unix time (seconds since epoch) representing time when object was created (if None, current time is used)
        """
        if timestamp is not None:
            return timestamp
        else:
            return time.time()
    
    def hash_obj(self, *args):
        """
        calculate hash of given arguments. arguments are byte encoded if necessary, concatenated, and hashed with SHA256.
        
        Parameters
        ----------
        *args : str
            an arbitrary number of string arguments
        """
        m = sha256()
            
        for a in args:
            if type(a) is bytes:
                m.update(a)
            elif type(a) is str:
                m.update(a.encode())
            else:
                raise Exception(f'arg {a} is wrong type: {type(a)}')
            
        return m.hexdigest()
        
    def display_creation(self, display):
        if display:
            time_struct = time.gmtime(self.timestamp)
            time_ascii = time.asctime(time_struct)
            
            print(f'{self.obj_type} creation time: {time_ascii}')
            print(f'{self.obj_type} hash: {self.hash_val}')

#### Defining Transactions

We're finally getting to the good stuff. Below define the `Transaction` class. We want hash values and timestamps for transactions, so `Transaction` extends `Hashable`. Outside of that, a transaction has a `sender`, a `recipient` and the `amount` being transacted.

The `sender` attribute is the public key of the account sending currency. We store the `ecdsa.keys.VerifyingKey` object. We **DO NOT** store the hexadecimal string representation of the public key here. If we did, we wouldn't have useful methods available to us, like `VerifyingKey.verify()`.

The `recipient` attribute is the public key of the account receiving the currency. We again use the `ecdsa.keys.VerifyingKey` object, for consistency.

The `amount` attribute is the amount of currency being sent by the `sender` to the `recipient`.

In order to calculate the hash we use four attributes. First we call the `to_string()` methods (but not `to_string().hex()` method) on the `sender` and `recipient`. Then we convert the `amount` (which is an `int`) to a `string`. We also convert `timestamp` from an `int` to a `string`. We hash their concatenation: `sha256(sender_str + recipient_str + amount_str + timestamp_str`). **Make sure to do it in that order**. `Hashable.hash_obj()` takes care of encoding the strings as bytestrings when necessary, and stores the hash as `hash_val`.

There is one last attribute to mention, `signature`. If the transaction is going to be included in the blockchain, it had better be digitally signed by the `sender`, otherwise how would we be able to verify that the `sender` actually approved the transaction? The `signature` attribute is the digital signature produced by `Wallet.sign_message()`. If you recall, it's a bunch of bytes. We could convert to a hexstring and store that, but there is no reason to; we'll store the raw bytes. The caveat is that we can't include the `signature` as a parameter to the initialization function. The `signature` is the digital signature produced by the `sender` signing the `hash_val`. That means we first have to instantiate the transaction object and hash it. Then we can pass `hash_val` to the wallet of the `sender` calling `sign_message`. The resulting `signature` can then be stored in the `Transaction` object.

**Fill in the code below, where it says TODO**.

In [74]:
class Transaction(Hashable):
    """
    A class that represents a transaction between sender and recipient. Subclass of Hashable.
    
    Attributes
    ----------
    sender : ecdsa.keys.VerifyingKey
        public key of the transaction's sender
    recipient : ecdsa.keys.VerifyingKey
        public key of the transaction's recipient
    amount : int
        amount being transacted
    timestamp : int
        Unix time (seconds since epoch) of transaction creation
    signature : bytes
        byte encoded digital signature, signed by private_key (via Wallet.sign_message()).
        message used for digital signature should be the transaction hash.
    """
    
    def __init__(self, sender, recipient, amount, timestamp=None, display=False):
        """
        initializes transaction with given parameters and hashes the transaction data (see Hashable.hash_obj()).
        hash is concatenation of sender, recipient, amount and timestamp.
        
        Parameters
        ----------
        sender : ecdsa.keys.VerifyingKey
            public key of the transaction's sender
        recipient : ecdsa.keys.VerifyingKey
            public key of the transaction's recipient
        amount : int
            amount being transacted
        timestamp : int
            Unix time (seconds since epoch) of transaction
        """
        
        """
        TODO: Your code here (hint: this is a subclass of Hashable, make use of super() for the timestamp)
        """
        
    def hash_transaction(self):
        """
        hash the transaction details (see Hashable.hash_obj())
        """
        
        """
        TODO: Your code here (hint: this is a subclass of Hashable, make use of hahs_obj())
        """
    
    def add_signature(self, signature):
        """
        set the digital signature of the transaction hash
        
        Parameters
        ----------
        signature : bytes
            byte encoded digital signature retrieved from private key of sender (see Wallet.sign_message())
        """
        self.signature = signature
    
    def display_creation(self, display):
        if display:
            super().display_creation(display)
            print(f'{self.obj_type} sender: {self.sender.to_string().hex()}')
            print(f'{self.obj_type} recipient: {self.recipient.to_string().hex()}')
            print(f'{self.obj_type} amount: {self.amount}')

We'll define some transactions that will be useful for us a bit later.

In [79]:
tx1 = Transaction(alice.public_key, bob.public_key, 50, tx1_ts, display=True)
tx2 = Transaction(alice.public_key, carol.public_key, 10, tx2_ts)

Transaction creation time: Mon Feb  1 00:00:00 2021
Transaction hash: 4f7963fcb8c88541c42f3e10964072f4b168569cc2ad71362c350a90fdea8d88
Transaction sender: 8ec3a2d3a8b406879ad54e78c83aca27c2875972cd48a922938a1e8b31d00a6db7e3fd2dc35cddc91fdba2f6c2d19795a375a0041026bcfce49fb15ef7841815
Transaction recipient: 0010b324b02beba152fd0f285e153f0cb153166e02fc32c5826f8fae8d6dbd2d54d6c5645ec2d14c29a20d2477f2bbe2b73b086fabec8d27bf98350ed8cf2163
Transaction amount: 50


#### Defining Blocks

The `Block` class is a bit more complex than a `Transaction`, but not much.

Firstly, `Block` extends `Hashable` so it has a hash stored in `hash_val` and a `timestamp`.

Recall that to add a block to the blockchain it must refer to the hash of a previous block. We have a `prev_hash` attribute for that. Easy enough, when creating a `Block` pass in the `hash_val` of the most recently mined block. But what if it's the first block to be mined and added to the blockchain? In that case we'll use the value:

`'0000000000000000000000000000000000000000000000000000000000000000'` (a string with $64$ $0$ values).

$64$ makes sense, but why a string of all $0$'s? Well it sort of arbitrary. But we know that to mine a block we have to beat a target difficulty, defined by the number of $0$'s it has. What's more difficult than all $0$'s?

For defining the `Block` class we don't have to worry about specific values of `prev_hash`, that will come a bit later down, but its good to introduce it now.

Next, we know a block should contain transactions. We'll create three different attributes related to transactions.

The first is `coinbase`. Recall that the `coinbase` is the reward to the miner. For us, we'll define a `coinbase` transaction as a regular `Transaction` object. The only thing special about it is that we create a special attribute to identify it. There is only one `coinbase`, and its seperated out from the rest of the transactions.

The next attribute is `transactions`. This is a list (array) of transactions to be included in the block. Just to emphasize, the `coinbase` is **not** in this list, it's stored separately. Why a list? Shouldn't we be using a merkle tree? Well, yes that's what Bitcoin does. But again I want to reduce the work for you and keep the complexity low to keep the assignment from growing too large.

The third transation-related attribute is the `transactions_hash`. Every `Transaction` object has the `hash_val` set. The `transactions` list is an array of `Transaction` objects, each with their own `hash_val`. To calculate the `transactions_hash` we **first** get the `hash_val` of the `coinbase`, **and then** iterate through the list, get the `hash_val` of each `Transaction` object, and concatenate them all together. Then we hash the concatenated string with `sha256()`. Remember, `hash_val` is stored as a hexadecimal string, so we need to byte encode the `hash_val` strings using `.encode()`. We call `hexdigest()` and store the result as `transactions_hash`. **It's important that you include the coinbase hash_val in the concatenation, and that it is first!**

Next are the `target` and `nonce` attributes. `target` is the difficulty setting for mining. Recall that when we mine a block, we calculate different hashes by iterating the `nonce` until the requisite number of $0$'s prefixing the `Block.hash_val` is reached. The `nonce` is a simple `int`, initialized to `0`. The `target` is a hexadecimal string of length $64$. But what does that have to do with the number of $0$'s? Well take this sample target:

In [87]:
sample_target = '00'+'f'*62
sample_target

'00ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff'

Notice that it has two $0$'s followed by $62$ $\text{f}$'s. Any hexadecimal number of length $64$ with two or more $0$'s will be less than or equal to `sample_target`. For example, take this sample hash (the value $31$ is the number of bytes):

In [110]:
sample_hash = '00'+os.urandom(31).hex()
sample_hash

'00b260eafbfd87e087e1bb6eba65de3b8f056141e21113cadeaa5913bef0e3b8'

Is it lower than `sample_target`? Yes! `'00ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff'` is the absolute largest (in value) hexadecimal number, of length $64$, with two $0$'s in front. Any hexadecimal number, of length $64$, that is less than or equal to `sample_target` will have two or more $0$'s in front.

But how do we do the comparison in code? Well we cast it into an integer using `int()`. You're probably used to passing a single argument to `int()`, but it actually takes a second argument as well. The second argument is the base. Hexadecimal is base $16$, so we pass that as the second argument:

In [111]:
int(sample_target, 16)

452312848583266388373324160190187140051835877600158453279131187530910662655

So for the comparsion, we just cast both to ints, specifing base $16$:

In [112]:
int(sample_hash, 16) <= int(sample_target, 16)

True

If we have a hash with a single $0$ in front (or fewer), the comparsion will return `False`:

In [115]:
sample_hash_bad = '01'+os.urandom(31).hex()

int(sample_hash_bad, 16) <= int(sample_target, 16)

False

What about the values of `nonce`? Won't we get different values of the `nonce` depending on our computer? No! Everything has been determinstic up to now, so as long as you iterate the `nonce` starting at `0` and increasing one by one, I know exactly what values of the `nonce` will produce the first `hash_val` below the `target`.

`Block` extends `Hashable`, so the next attribute of `Block` is `timestamp`. You should know all about this by now. What about `hash_val`? What do we include? We'll include a concatenation of the `nonce` (cast as a string), `previous_hash`, `timestamp` (cast as a string), and `transactions_hash`. **Make sure to do it in that order.**

The final attribute is `mine_time`. This is the amount of time, in seconds, it took to mine the block. We obviously dont pass in `mine_time` as an argument to the intializing function. Instead we pass in the `mine` argument, as a boolean. If `mine` is set to `False`, no mining occurs. This is useful for verifying blocks as we'll see in a bit. If `mine` is set to `True` the mining process begins on initialization; the `nonce` is iterated, and `hash_val` is re-calulated until it is below the `target`. After mining is complete, set the `mine_time` to the number of seconds it took. We've been keeping everything determinstic up to now, but the values of `mine_time` will differ since your computer will take a different amount of time to mine a block than mine. That's okay. We don't need specific values for `mine_time`.

**Fill in the code below, where it says TODO.**

In [124]:
class Block(Hashable):
    """
    A class that represents a Block. Subclass of Hashable.
    A Block includes collection of transactions (including coinbase).
    Most times a Block is mined, but sometimes a shell Block is created where no mining occurs for verification.
    
    Attributes
    ----------
    previous_hash : length 64 string encoding of hexadecimal number (see Hashable.hash_val)
        a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
        represents hash_val of previous block
    coinbase : Transaction
        a transaction where the recipient is set to the miner of the block (rewarded with mining fee, see Blockchain.mining_reward)
    transactions : list[ Transaction ], may be empty list
        a list of transactions to be included in the block; should be retrieved from transaction pool (see Blockchain.get_n_transactions())
    transactions_hash : length 64 string encoding of hexadecimal number (see Hashable.hash_val)
        a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
        represents hash derived from concatenation of all individual transaction hashes
    target : length 64 string encoding of hexadecimal number (see Blockchain.target)
        a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
        Block.hash_val must be less than or equal to this number, which is achieved in mining process
    timestamp : int
        Unix time (seconds since epoch) of block creation
    nonce : int
        value to iterate in order to generate random hash_vals
        nonce derived from mining processs and has property such that concatenation with Block data produces hash less than or equal to target
    hash_val : length 64 string encoding of hexadecimal number (see Hashable.hash_val)
        a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
    mine_time : int
        number of seconds it took to find ideal nonce value
    """
    
    def __init__(self,
                 previous_hash,
                 coinbase_transaction,
                 transactions,
                 target,
                 timestamp=None,
                 mine=True,
                 display=False):
        """
        initializes block with data, and then starts mining (if specified) to find ideal nonce
        
        Parameters
        ----------
        previous_hash : length 64 string encoding of hexadecimal number (see Hashable.hash_val)
            a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
        coinbase_transaction : Transaction
            a transaction where the recipient is set to the miner of the block (rewarded with mining fee, see Blockchain.mining_reward)
        transactions : list[ Transaction ], can be empty list
            a list of transactions to be included in the block; should be retrieved from transaction pool (see Blockchain.get_n_transactions())
        target : length 64 string encoding of hexadecimal number (see Blockchain.target)
            a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
            Block.hash_val must be less than or equal to this number, which is achieved in mining process
        timestamp : int, optional
            Unix time (seconds since epoch) of block creation (default is current time)
        mine : bool
            specifies whether the mining process should be initialized (default True)
            mine=False useful for creating shell block in order to use in verification process (see Blockchain.verify_block())
        """

        """
        TODO: your code here (hint: Block is a subclass of Hashable, make sure to use super() for timestamp)
        """
        
        self.mine_block(mine)
        self.display_creation(display)
    
    def hash_transactions(self, coinbase_transaction, transactions):
        """
        calculate hash of all transactions by hashing the concatenation of each individual transaction hash
        
        Parameters
        ----------
        coinbase : Transaction
            a transaction where the recipient is set to the miner of the block (rewarded with mining fee, see Blockchain.mining_reward)
        transactions : list[ Transaction ], may be empty list
            a list of transactions to be included in the block; should be retrieved from transaction pool (see Blockchain.get_n_transactions())
        """

        """
        TODO: your code here (IMPORTANT: the coinbase transaction should go first in the concatenation,
                              hint: don't use hash_obj() directly, but the code is similar)
        """
  
    def hash_block(self):
        """
        calculate hash of the block by hashing concatenation of nonce, the previous block hash, block creation timestamp, and hash of transactions
        """
        
        """
        TODO: your code here (hint: Block is a subclass of Hashable, use hash_obj())
        """
    
    def mine_block(self, mine=True):
        """
        mine the block by iterating nonce and hashing block. once block hash less than or equal to target, stop and log time it took in seconds
        
        Parameters
        ----------
        mine : bool
            specifies whether mining should occur
        """
        
        if mine:
            """
            TODO: your code here (hint: use time.time() for the start and end time (similar to memoization in hw 0))
            """
            self.mine_time = end - start # in seconds
    
    def display_creation(self, display):
        if display:
            super().display_creation(display)
            print(f'{self.obj_type} miner: {self.coinbase.sender.to_string().hex()}')
            print(f'{self.obj_type} reward: {self.coinbase.amount}')
            print(f'{self.obj_type} nonce: {self.nonce}')
            print(f'{self.obj_type} previous hash: {self.previous_hash}')
            print(f'{self.obj_type} mine time: {self.mine_time:.6f} seconds')

We won't define any blocks quite yet. We'll do that futher down.

#### Defining the Blockchain

Okay, now let's define the `Blockchain` class. It's a bit bigger than the previous ones, but you can handle it. A `Blockchain` instance stores the state of the blockchain. It also is responsible for verifying information. Any blocks or transactions added must be validated by methods defined in `Blockchain`.

`Blockhain` stores the chain of blocks (suprise). The `chain` attribute is a simple list (array), that contains instances of `Block`. A `Block` object is appended to `chain` only after it has been verified. We'll discuss that in just a bit.

`Blockchain` also contains a transaction pool. In any given Blockchain, peers submit transactions to be included in blocks. But they don't send the transactions to miners over email, or through direct message. They place their transaction in the transaction pool. Miners will then build a block by taking some transactions out of the transaction pool, putting them in the block, and then mining the block. In reality miners choose transactions based on the miner fees those tranasations pay. The higher the amount a sender is willing to give to the miner, the more likely their transaction will be picked up first. We're not implementing fees, so we'll ignore that. The `transaction_pool` attribute is a list (array) of `Transaction` objects.

Remember our `Account_Registry`? We bind it to our `Blockchain` instance in the `account_registry` attribute. We can either pass in an instance of `Account_Registry` or if not provided, instantiate a new one.

The `Blockchain` instance is also instantiated with a `target` attribute. Didn't the `Block` class also have this? Yes, but we want one here to. The `Block.target` attribute is used during the mining process. The `Blockchain.target` attribute is used for verification. It says that all `Block` objects included in the `Blockchain` instance must have their targets below `Blockchain.target`. The initialization function doesn't take in a hex string for the target. Instead it uses a `num_zero_difficulty` argument, which is an `int`. This is the number of zeros that should prefix a block's `hash_val`. In other words the `target` is defined by the number of zeros set by `num_zero_difficulty`. The `target` is set to `num_zero_difficult` $0$'s, followd by however many $\text{f}$'s it takes to reach a hexadecimal string of length $64$. I demonstrated that above, so hopefully it's fairly straightforward.

Finally there is the `mining_reward` attribute. All `Block.coinbase` transactions must have the `coinbase.amount` set to be less than or equal to `mining_reward`. If a block's `coinbase` transaction has an `amount` greater than `mining_reward`, then the block can't be included in the `chain`.

That covers the attributes and the initialization function. Let's discuss some of the methods of `Blockchain`.

`get_target()` is easy, it retrieves the set `target`.

`get_last_hash()` returns the hash value of the most recently added block. What if no blocks have been added yet? We discussed that above. It should return a string of lenght $64$, with all $0$'s.

`verify_transaction()` is our first verification method. How do we verify a transaction? A transaction is valid if it is signed (`Transaction.signature` is set), and the `sender` can validate the signature. Remember `sender` is a public key of type `ecdsa.keys.VerifyingKey`. We already have the verifying key right there. We also have the `Transaction` object, which contains both the `signature` attribute and the `hash_val` attribute which was the original message. All we need to do is call `verify` passing in the relevent information.

`add_transaction()` adds a tranasction to the `transaction_pool`. It must first verify the transaction (hint: we just described that above). If it's valid, append it to the `transaction_pool`.

`add_transactions()` is very similar. It takes in a list (array) of `Transaction` objects, validates all of them, and add each of them to the transaction pool. It should be dead simple, since we already defined the main components above.

`get_n_transactions()` takes in `num` as an argument, and returns `num` transactions from the `transaction_pool`. It's important to emphasize here, that the transactions are **removed** from the transaction pool. Don't just reference them, take them out so they are no longer in the pool.

`verify_block()` is our second main verification method. How do we know a block is valid? Well first off, a `Block` object has a `coinbase` attribute contianing a transaction. The `amount` specified in that transaction should be less than or equal to `mining_reward`. In other words check that the miner has not rewarded themselves too much. Next we have to check the `hash_val` of the `Block` instance. This is where the `mine` argument for `Block.__init__()` is useful. We construct a shell block, meaning we set `mine=False` and construct the block using the attributes of the given block (including the given `nonce`). Then we just compare the `hash_val` of the shell block to the `hash_val` of the provided block. We also have to make sure the `hash_val` of the provided block is less than the `target`. But which `target`? Should we use `Block.target` or `Blockchain.target`. Well we're verifying the provided block, so we can't trust it. It's probably better to use `Blockchain.target`. Do the comparison by following the demonstration I made above (cast to `int` with base $16$). Finally, notice the provided block has a list of transactions in it. Verify all of them.

`add_block()` adds a block to the `chain` list. You will want to verify the block before appending it to `chain`.

`verify_chain()` verifies a whole chain of blocks. I'll discuss why that's useful in just a bit, but the process is easy. A chain of blocks is a list (array) of `Block` objects. Use `verify_block()` on them.

`find_block_idx()` takes in the hash of a block as an argument. It then searches `chain` and finds a block with a matching hash value (if it exists). It returns the index of that block. Therefore `chain[ block_index ]` returns a `Block` object with a `hash_val` matching the hash provided as an argument.

`add_subchain()` is the reason `verify_chain()` and `find_block_idx()` are useful. It takes in a list (array) of `Block` objects. This list represents a subchain. We want to attach this subchain somewhere on the existing `chain`. The first block in the subchain has a `prev_hash` attribute (as all blocks do). That `prev_hash` points to a block somewhere in the existing `chain`. Find the index of this block. Create a new chain that consists of all blocks up to and including the block pointed to be `prev_hash`, followed by all the blocks in the subchain. This new chain needs to be verified. Do that using `verify_chain()`. Finally, it only makes sense to replace the existing `chain` with the new chain if the new chain is longer than the existing one. That's easy. Whichever one has more blocks wins. If they are equal in length, keep the existing chain and discard the new one.

**Fill in the code below, where it says TODO**.

In [120]:
class Blockchain:
    """
    A class that represents a chain of blocks, and associated methods to modify or query that chain.
    Contains transaction pool, and associated methods to retrieve transactions from pool.
    Contains Blockchain parameters such as reward and mining difficulty (target).
    Contains registry of accounts.
    Contains methods to verify additions to blockchain.
    
    Attributes
    ----------
    chain : list[ Block ]
        a list of Blocks in order of when they were mined (increasing value of block timestamps)
    transaction_pool : list[ Transaction ]
        a list of transactions waiting to be included in blocks
    account_registry : Account_Registry
        a registry of accounts associated with the blockchain, public keys associated with a name, balance, and wallet
    target : length 64 string encoding of hexadecimal number
        a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
        Block.hash_val must be less than or equal to this number, which is achieved in mining process (see Blockchain.verify_block())
    mining_reward : int
        reward for successfully mining a block and including in blockchain (default is 100)
    """
    
    def __init__(self, account_registry=None, num_zero_difficulty=1, reward_amount=100, display=False):
        """
        initializes blockchain by creating empty chain and transaction pool, and account registry, as well as blockchain settings.
        
        Parameters
        ----------
        account_registry : Account_Registry
            public keys associated with a name, balance, and wallet (default is None, where empty account registry is created)
        num_zero_difficulty : int
            difficulty setting for mining process, specified by number of leading 0's, used in determining target.
            all blocks added to chain must have hash values with as many leading 0's.
            must be between 1 and 64.
            default is None, in which case 1 leading 0 is used, followed by 63 F values.
        reward_amount : int
            amount to reward a miner of a block
        """
        
        """
        TODO: your code here - initialize the chain and transaction_pool, set target and reward_amount
        """
        
        if account_registry and isinstance(account_registry, Account_Registry):
            self.account_registry = account_registry
        else:
            self.account_registry = Account_Registry()
        
    def get_target(self):
        """
        retrieve target (difficulty). hash of a block must be less than this value in order to be included in blockchain.
        """
        return self.target
        
    def get_last_hash(self):
        """
        get the hash of the most recently verified block added to blockchain.
        if no blocks have been added yet, hexstring of all '0' values is returned.
        """
        
        """
        TODO: your code here - return the hash of the most recent block, or the default if none have been added yet
        """
    
    def verify_transaction(self, transaction, display=False):
        """
        verify a transaction.
        for a transaction to be valid:
            its signature attribute must be set (via Transaction.add_signature())
            it must be set to the digital signature (via Wallet.sign_message()) of the transaction hash (Transaction.hash_val)
            the transaction sender (ecdsa.keys.VerifyingKey) must be able to verify the signature (ecdsa.keys.VerifyingKey.verify())
            
        Parameters
        ----------
        transaction : Transaction
            the transaction to verify, with the signature attribute set (via Transaction.add_signature())
        """
        if not isinstance(transaction, Transaction):
            raise Exception(f'tx "{transaction}" is not of type "Transaction"')
            
        if not isinstance(transaction.sender, VerifyingKey):
            raise Exception(f'tx sender "{transaction.sender}" not of type "ecdsa.VerifyingKey"')

        if not isinstance(transaction.recipient, VerifyingKey):
            raise Exception(f'tx recipient "{transaction.recipient}" not of type "ecdsa.VerifyingKey"')
            
        if transaction.sender.to_string().hex() not in self.account_registry.accounts:
            raise Exception(f'tx sender {transaction.sender.to_string().hex()} not registered')
            
        if transaction.recipient.to_string().hex() not in self.account_registry.accounts:
            raise Exception(f'tx recipient {transaction.recipient.to_string().hex()} not registered')
            
        """
        TODO: your code here - verify the transaction signature by making sure the sender can produce the correct digital signature; error if not
        """
            
    def add_transaction(self, transaction, display=False):
        """
        verify a transaction (Blockhain.verify_transaction()) and add it to the transaction pool
        """
        
        """
        TODO: validate the transaction and add it to the transaction pool
        """
        
        if display:
            sender_name = self.account_registry.accounts[transaction.sender.to_string().hex()]['name']
            recipient_name = self.account_registry.accounts[transaction.recipient.to_string().hex()]['name']
            print(f'tx sender: {sender_name} ({transaction.sender.to_string().hex()})')
            print(f'tx recipient: {recipient_name} ({transaction.recipient.to_string().hex()})')
            print(f'tx amount: {transaction.amount}')
        
    def add_transactions(self, transactions, display=False):
        """
        verify (Blockhain.verify_transaction()) and add (Blockchain.add_transaction()) a list of transactions to the transaction pool
        """
        
        """
        TODO: your code here - add transactions in provided list to the transaction pool
        """
        
    def get_n_transactions(self, num):
        """
        remove (up to) specified number of transactions from the transaction pool and return as a list.
        
        Parameters
        ----------
        num : int
            the number of transactions to retrieve from transaction pool
            if num > len(transaction_pool), then len(transaction_pool) number of transactions returned
        """
        
        """
        TODO: your code here - get num transactions from pool (hint: this mutates the transaction pool)
        """
    
    def verify_block(self, block):
        """
        verify a given block.
        for a block to be valid:
            the coinbase must be less than or equal to the mining reward (Blockchain.mining_reward)
            the attributes of the provided block (including its nonce value) should produce a valid hash
                this is verified by constructing a shell Block (mine=False), hashing it, and comparing the hash against the provided block
            the hash of the provided block must be less than the target (Blockchain.target)
            the individual transactions included in the block must be valid (Blockchain.verify_transaction())
            
        Parameters
        ----------
        block : Block
            a valid block
        """
        if not isinstance(block, Block):
            raise Exception(f'block "{block}" is not of type "Block"')
            
        """
        TODO: your code here - validate the block
                                   check coinbase against mining_reward,
                                   check hash_val of Block using given attributes (including nonce)
                                   verify transactions in Block (including coinbase)
                                   verify that the hash_val is below target
        """            
    
    def add_block(self, block):
        """
        verify (Blockchain.verify_block()) and add a block to the chain
        
        Parameters
        ----------
        block : Block
            a valid block
        """
        
        """
        TODO: your code here: validate the block and add it to the chain
        """

    def verify_chain(self, chain):
        """
        verify a chain of blocks, where each block must be valid (Blockchain.verify_block())
        
        Parameters
        ----------
        chain : list[ Block ]
            a list of valid blocks in order of creation (by timestamp)
        """
        
        """
        TODO: your code here - validate all blocks in provided chain
        """
            
    def find_block_idx(self, block_hash):
        """
        find the index of a block matching the provided block hash
        
        Parameters
        ----------
        block_hash : length 64 string encoding of hexadecimal number (see Hashable.hash_val)
            a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
        """
        
        for i, block in enumerate(chain):
            """
            TODO: your code here - find index of block in chain matching provided block_hash
            """
        raise Exception(f'could not find block with hash value {append_block.hash_val}')
    
    def add_subchain(self, subchain):
        """
        overwrite the current blockchain by attaching a subchain (if valid).
        the first block in the subchain (oldest timestamp) is queried for the hash of the previous block (Block.prev_hash),
        the subchain is then attached to the specified previous block, overwriting the current chain of blocks.
        the new chain formed by attaching the subchain is considered valid if:
            all blocks in the new chain must be valid (Blockhain.verify_chain())
            the size of the new chain must be greater than the size of the current chain
            
        Paramters
        ---------
        subchain : List[ Block ]
            list of blocks where the first block's prev_hash attribute is set to some block that is already in the current chain
        """
        
        """
        TODO: your code here - create new chain by appending it to prev_hash of first block in subchain,
                               validate new chain,
                               overwrite existing chain with new chain if longer, error otherwise
        """
            
    def display_blockchain(self):
        """
        display all blocks and the transactions they contain
        """
        for i, block in enumerate(self.chain):
            print(f'Block {i}')

            block_time_struct = time.gmtime(block.timestamp)
            block_time_ascii = time.asctime(block_time_struct)
            
            print(f'Block {i} creation time: {block_time_ascii}')
            print(f'Block {i} hash: {block.hash_val}')
            miner = block.coinbase.sender.to_string().hex()
            miner_name = self.account_registry.accounts[miner]['name']
            print(f'Block {i} miner: {miner_name} ({miner})')
            print(f'Block {i} reward: {block.coinbase.amount}')
            print(f'Block {i} nonce: {block.nonce}')
            print(f'Block {i} previous hash: {block.previous_hash}')
            print(f'Block {i} mine time: {block.mine_time:.6f} seconds')
            
            
            for ii, tx in enumerate(block.transactions):
                print(f'Block {i} - Transaction {ii}')
                
                tx_time_struct = time.gmtime(tx.timestamp)
                tx_time_ascii = time.asctime(tx_time_struct)
                
                print(f'Block {i} - Transaction {ii} creation time: {tx_time_ascii}')
                print(f'Block {i} - Transaction {ii} hash: {tx.hash_val}')
                sender = tx.sender.to_string().hex()
                sender_name = self.account_registry.accounts[sender]['name']
                print(f'Block {i} - Transaction {ii} sender: {sender_name} ({sender})')
                recipient = tx.recipient.to_string().hex()
                recipient_name = self.account_registry.accounts[recipient]['name']
                print(f'Block {i} - Transaction {ii} recipient: {recipient_name} ({recipient})')
                print(f'Block {i} - Transaction {ii} amount: {tx.amount}')

Finally, we'll need to sign transactions in order to set a `Transaction` object's `signature` field, I've created a helper function `sign_transaction()` which lets us do that.

In [121]:
def sign_transaction(wallet, transaction):
    """
    a transaction's signature attribute must be set before including in a block on the blockchain.
    this function produces a digital signature of the given transaction with the private key contained in the given wallet.
    the message used is the transaction hash.
    
    Parameters
    ----------
    wallet : Wallet
        wallet containing the private key (ecdsa.keys.SigningKey) with which to sign (Wallet.sign_message()) the transaction
    transaction : Transaction
        the transaction to sign and set (Transaction.add_signature())
    """
    if not isinstance(wallet, Wallet):
        raise Exception(f'wallet "{wallet}" not of type "Wallet"')
        
    if not isinstance(transaction, Transaction):
        raise Exception(f'tx "{transaction}" not of type "Transaction"')
        
    transaction.add_signature(wallet.sign_message(transaction.hash_val))

Whew! We're done with our definitions. Now we can see it in action. First let's instantiate a blockchain. We'll pass in the `account_registry` we've already created. We'll set the difficulty to $2$ zeroes.

In [126]:
blockchain = Blockchain(account_registry=account_registry, num_zero_difficulty=2, display=True)

Now we'll greate the first block. This is known as the **genesis block**. We'll assign it to Nakamoto. We create a wallet for Nakamoto. Then we register his or her account in the account registry. We create a coinbase transaction as a reward for mining the first block. Importantly, we have to sign the transaction. We use the `sign_transaction()` helper from above. We create the block. It's the first block, so we set the `previous_hash` to an all $0$ valued hexstring as returned by `get_last_hash()`. We won't include any other transactions. Finally, we add the block using `add_block()`.

In [128]:
nakamoto = Wallet(seed_hex='6daa908eb3d309183a9f4789a00f51b18542da1417be0cdd799081228236dae9', display=True)
blockchain.account_registry.register_account(nakamoto, 'nakamoto')
genesis_coinbase_transaction = Transaction(nakamoto.public_key, nakamoto.public_key, blockchain.mining_reward, genesis_timestamp, display=display)
sign_transaction(wallet=nakamoto, transaction=genesis_coinbase_transaction)

genesis_block = Block(previous_hash=blockchain.get_last_hash(),
                      coinbase_transaction=genesis_coinbase_transaction,
                      transactions=[],
                      target=blockchain.target,
                      timestamp=genesis_timestamp,
                      mine=True,
                      display=display)

blockchain.add_block(genesis_block)

Wallet seed: 6daa908eb3d309183a9f4789a00f51b18542da1417be0cdd799081228236dae9
Wallet secret exponent: 85314299752260761361628249051057399690231672481648748497211137228769388769381
Wallet private key: bc9e2eb5d3a6857f52803f57750a2e7c9a75e8d1a14784e61f38845ecc6db065
Wallet public key: 57b28b2862c10eb35a0cc9799543ad75ed88c1151396993e5d43bb2f1851acd2665e92bff0daf899374c1bf6723b84d112bdfed23ace5fa2f00e629080f7d638
Transaction creation time: Mon Jan 25 00:00:00 2021
Transaction hash: 61cee37ebe97ea40aab267dcd1d14ce9da3098f314b1bcb826534945a837e194
Transaction sender: 57b28b2862c10eb35a0cc9799543ad75ed88c1151396993e5d43bb2f1851acd2665e92bff0daf899374c1bf6723b84d112bdfed23ace5fa2f00e629080f7d638
Transaction recipient: 57b28b2862c10eb35a0cc9799543ad75ed88c1151396993e5d43bb2f1851acd2665e92bff0daf899374c1bf6723b84d112bdfed23ace5fa2f00e629080f7d638
Transaction amount: 100
Block creation time: Mon Jan 25 00:00:00 2021
Block hash: 00876534b9fb3d471dc0bf05ad2742dda3eacab68555cec37d72a0822ff3958a
Bloc

We created some transactions earlier, `tx1` and `tx2`. Let's have alice sign them since she's the sender for both.

In [129]:
sign_transaction(wallet=alice, transaction=tx1)

sign_transaction(wallet=alice, transaction=tx2)

Now we'll add them to the transaction pool using `add_transactions()`.

In [130]:
blockchain.add_transactions([tx1, tx2], display=True)

tx sender: alice (8ec3a2d3a8b406879ad54e78c83aca27c2875972cd48a922938a1e8b31d00a6db7e3fd2dc35cddc91fdba2f6c2d19795a375a0041026bcfce49fb15ef7841815)
tx recipient: bob (0010b324b02beba152fd0f285e153f0cb153166e02fc32c5826f8fae8d6dbd2d54d6c5645ec2d14c29a20d2477f2bbe2b73b086fabec8d27bf98350ed8cf2163)
tx amount: 50
tx sender: alice (8ec3a2d3a8b406879ad54e78c83aca27c2875972cd48a922938a1e8b31d00a6db7e3fd2dc35cddc91fdba2f6c2d19795a375a0041026bcfce49fb15ef7841815)
tx recipient: carol (4ccb71339d78b7bb04417f04c7dc50d47b30d04568105e69e69490dd06d223793e20a090a9cbcd1743ac7b5abfe0e76cea4cbd105e331447232a32c9cd6a4838)
tx amount: 10


Here we use our imagination. Some other miner on the network retrieves the transactions from the transaction pool using `get_n_transactions()`.

In [131]:
b1_txs = blockchain.get_n_transactions(2)

For the miner to include these two transactions in a block, we need the `prev_hash` value. Right now that refers to the genesis block.

In [132]:
genesis_block_hash = blockchain.get_last_hash()

We'll have alice mine this block. Therefore she creates a coinbase transaction rewarding herself. The timestamp for block 1 is `b1_ts`, which we define earlier. We sign the coinbase transaction using alice's wallet. Then we build and mine the block.

In [ ]:
b1_coinbase_tx = Transaction(alice.public_key, alice.public_key, blockchain.mining_reward, b1_ts, display=True)
sign_transaction(wallet=alice, transaction=b1_coinbase_tx)

block1 = Block(
    genesis_block_hash,
    b1_coinbase_tx,
    b1_txs,
    blockchain.target,
    timestamp=b1_ts,
    mine=True,
    display=True)

Now that the block is mined, we'll add it to the blockchain. Again using our imagination, every node on the network has an instance of Blockchain set to the same initialization parameters. They will all receive a copy of the block, verify it, and add it using `add_block()`.

In [134]:
blockchain.add_block(block1)

Here we can display all the blocks and transactions up to this point.

In [ ]:
blockchain.display_blockchain()

### Submission Instructions

Upload the completed notebook to canvas. When you submit the assignment leave a comment. The comment should be the "hash_val of block1 is:" followed by the actual hash of block1. If you did everything right, you'll get an exact value. **You must submit this hash to get full points**. If you include a hash, but its wrong, you will get partial points. I will look through the code you wrote in the submitted notebook and give points for good effort. If you submit the completed notebook and the correct hash value, you are gauranteed full points.

**Do not submit a separate python file**. You must submit this notebook, completed, and nothing else (apart from the comment). Do not import any libraries that have not already been imported in the first code cell of this notebook. If you want to modify any of the existing code that I have provided, make a copy and do it below here (or just use a separate notebook, but don't submit it). Don't modify what I've provided outside of the TODO sections (unless you are implementing the bonus below).


#### Bonus (1 point)

You'll notice I haven't implemented any code that checks or modifies the account balance contained in the account registry. Implement this feature. When a transaction from Alice is sent to Bob with the amount of 10. It should decrease Alice's account by 10, and increase Bob's by 10. If Alice doesn't have 10, it should throw an error. When miners mine blocks, their balanace should go up by the reward amount.

If you implement this, include an additional comment saying "I have implemented the bonus".

Note: the submitted hash for block 1 should remain the same. Implementing the account balanace feature should not change the hashes of blocks or transactions.


#### Note on final project

After submitting, feel free to play around with this notebook. Break things, exploit the many security holes I've left. If you enjoy it, consider extending it for your final project. Modify the existing code or start from scratch (I won't be offended). If you decide to do that, it should be a bit more sophisticated than what's in this notebook. Some ideas:
- Use threads to simulate a distributed network where multiple miners are competing
- Do the above and simulate an attack, e.g. > 51% attack
- Replace the proof of work mechanism with proof of stake (or some other system)
- Modify the existing code to be pure UTXO
- Implement a simple scripting mechanism similar to bitcoin
- Whatever you want

Most final projects will implement smart contracts on an Ethereum-like network. But I do like to see simulation-oriented projects, so if you have an interest, feel free.